In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import random
import cv2

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
train_dir = '../input/beauty-detection-data-set/train/'
test_dir = '../input/beauty-detection-data-set/test/'

In [ ]:
Name=[]
for file in os.listdir(train_dir):
    Name+=[file]
print(Name)
print(len(Name))

In [ ]:
N=[]
for i in range(len(Name)):
    N+=[i]
    
normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 

In [ ]:
trainx0=[]
trainy0=[]
count=0
for file in Name:
    path=os.path.join(train_dir,file)
    for im in tqdm(os.listdir(path)):
        image=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(48,48))
        image=img_to_array(image)
        image=image/255.0
        trainx0.append(image)
        trainy0.append(count)
    count=count+1

In [ ]:
testx0=[]
testy0=[]
count=0
for file in Name:
    path=os.path.join(test_dir,file)
    for im in tqdm(os.listdir(path)):
        image=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(48,48))
        image=img_to_array(image)
        image=image/255.0
        testx0.append(image)
        testy0.append(count)
    count=count+1

In [ ]:
trainy2=to_categorical(trainy0)
X_train=np.array(trainx0).reshape(-1,48,48,3)
y_train=np.array(trainy2)

In [ ]:
X_test=np.array(testx0).reshape(-1,48,48,3)

In [ ]:
trainx,testx,trainy,testy=train_test_split(X_train,y_train,test_size=0.2,random_state=44)

In [ ]:
print(trainx.shape)
print(testx.shape)
print(trainy.shape)
print(testy.shape)

In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=20,
                zoom_range=0.8,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.1,
                fill_mode="nearest")

In [ ]:
pretrained_model3 = tf.keras.applications.DenseNet201(input_shape=(48,48,3),
                                include_top=False,weights='imagenet',pooling='avg')
pretrained_model3.trainable = False

In [ ]:
inputs3 = pretrained_model3.input
x3 = tf.keras.layers.Dense(128, activation='relu')(pretrained_model3.output)
outputs3 = tf.keras.layers.Dense(2, activation='softmax')(x3)
model = tf.keras.Model(inputs=inputs3, outputs=outputs3)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
his=model.fit(datagen.flow(trainx,trainy,batch_size=32),validation_data=(testx,testy),epochs=20)

In [ ]:
y_pred=model.predict(testx)
pred=np.argmax(y_pred,axis=1)
ground = np.argmax(testy,axis=1)
print(classification_report(ground,pred))

In [ ]:
get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']

epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training vs validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training vs validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
path0='../input/beauty-detection-data-set/test/beautiful/016.jpg'
load_img(path0,target_size=(300,300))

In [ ]:
image=load_img(path0,target_size=(48,48))

image=img_to_array(image) 
image=image/255.0
prediction_image=np.array(image)
prediction_image= np.expand_dims(image, axis=0)

In [ ]:
prediction=model.predict(prediction_image)
value=np.argmax(prediction)
move_name=reverse_mapping[value]
print("Prediction is {}.".format(move_name))

In [ ]:
pred2=model.predict(X_test)
print(pred2[0:10])

PRED=[]
for item in pred2:
    value2=np.argmax(item)      
    PRED+=[value2]
print(PRED[0:10])

In [ ]:
ANS=list(testy0)
ANS[0:10]

In [ ]:
pd.Series(PRED).value_counts()

In [ ]:
accuracy=accuracy_score(ANS,PRED)
print(accuracy)

In [ ]:
id0sample=random.sample(range(len(testx0)),k=9)

In [ ]:
fig,axs = plt.subplots(3,3,figsize=(14,14))
for i in range(9):
    image_input = testx0[id0sample[i]]
    r=i//3
    c=i%3
    axs[r][c].set_xticks([])
    axs[r][c].set_yticks([])
    axs[r][c].set_title('Pred:'+reverse_mapping[PRED[id0sample[i]]]+', Ans:'+reverse_mapping[ANS[id0sample[i]]])  
    ax=axs[r][c].imshow(cv2.cvtColor(image_input, cv2.COLOR_BGR2RGB))
plt.show()